In [1]:
import seaborn as sns

In [21]:
import plotly.express as px
import pandas as pd
import numpy as np
import pymcdm
np.set_printoptions(suppress=True, precision=4) 

validation_df = pd.read_csv("analysis/data/validation_df_epoch99.csv")
df = pd.read_csv("analysis/results_df_find_num_neurons.csv")
df = pd.read_csv("analysis/data/validation_df_epoch99.csv")
df["activation_name"] = df["activation"]
df = df.drop(["activation"], axis=1)
df.head()
# grouped_df = df.groupby(["activation_name", "architecture_id"]).agg(["mean", "std"]).reset_index()
# grouped_df.head()


,Unnamed: 0,architecture_id,num_neurons,regularization_term,training_loss,validation_loss,model_id,start_epoch,end_epoch,train_aucroc_micro,...,valid_overall_acc,test_aucroc_micro,test_aucroc_macro,test_aucroc_weighted,test_aucroc_samples,test_matthews_corrcoef,test_f1_macro,test_f1_micro,test_overall_acc,activation_name
0,0,0,1,6.732450,0.657658,0.658622,0,0.0,99,0.566923,...,0.641026,0.555346,0.500000,0.475808,0.666667,0.330798,0.664127,0.666667,0.666667,Identity()
1,1,1,2,4.426847,0.482172,0.491109,1,0.0,99,0.891541,...,0.801282,0.894560,0.887179,0.887737,0.840580,0.677176,0.838407,0.840580,0.840580,Identity()
2,2,2,3,4.634713,0.407352,0.397975,2,0.0,99,0.910533,...,0.878205,0.927326,0.925214,0.924935,0.898551,0.794724,0.897168,0.898551,0.898551,Identity()
3,3,3,4,4.008699,0.405150,0.405159,3,0.0,99,0.917063,...,0.871795,0.929637,0.924359,0.924415,0.913043,0.823077,0.911538,0.913043,0.913043,Identity()
4,4,4,5,4.244227,0.413947,0.429383,4,0.0,99,0.900895,...,0.820513,0.914304,0.908120,0.908621,0.898551,0.794368,0.895387,0.898551,0.898551,Identity()


In [4]:
# px.violin(grouped_df, x="num_neurons", y="loss", orientation="h").update_traces(side="positive", width=4)
# px.scatter_3d(grouped_df, x=("num_neurons", "mean"), y="activation_name", z=("loss", "mean"))
df.groupby("architecture_id").median().head().sort_values("loss")

grouped_df.head().sort_values(( "loss", "mean"))
df.groupby(["activation_name", "architecture_id"]).agg(["mean", "min", "std", "count", lambda x: x.quantile(0.95)]).reset_index().sort_values(by=("loss", "<lambda_0>"))[("model_id","count")].values[0]



20

In [71]:
# validation_df.groupby("architecture_id").mean().sort_values(by="test_overall_acc", ascending=True)
# validation_df.describe()

def is_pareto_efficient(costs, return_mask = True):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array of costs to be minimized
    :param return_mask: True to return a mask
    :return: An array of indices of pareto-efficient points.
        If return_mask is True, this will be an (n_points, ) boolean array
        Otherwise it will be a (n_efficient_points, ) integer array of indices.
    """
    is_efficient = np.arange(costs.shape[0])
    n_points = costs.shape[0]
    next_point_index = 0  # Next index in the is_efficient array to search for
    while next_point_index<len(costs):
        nondominated_point_mask = np.any(costs<costs[next_point_index], axis=1)
        nondominated_point_mask[next_point_index] = True
        is_efficient = is_efficient[nondominated_point_mask]  # Remove dominated points
        costs = costs[nondominated_point_mask]
        next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
    if return_mask:
        is_efficient_mask = np.zeros(n_points, dtype = bool)
        is_efficient_mask[is_efficient] = True
        return is_efficient_mask
    else:
        return is_efficient

tmp_df= pd.DataFrame()

grouped_df = validation_df.groupby(["architecture_id"]).agg(["mean", "median", "std", "count"]).reset_index()
tmp_df["num_neurons"] = grouped_df[("num_neurons", "mean")]
# grouped_df = validation_df.groupby(["num_neurons"]).agg(["mean", "median", "std", "count"]).reset_index()
# tmp_df["num_neurons"] = grouped_df["num_neurons"]
tmp_df["activation"] = validation_df.groupby("architecture_id").agg({"activation": "first"})
for k in ["mean", "std", "median"]:
	tmp_df[f"{k}_regularization_term"] = grouped_df[("regularization_term", k)]
	tmp_df[f"{k}_training_loss"] = grouped_df[("training_loss", k)]
	tmp_df[f"{k}_validation_loss"] = grouped_df[("validation_loss", k)]
	tmp_df[f"{k}_train_overall_acc"] = grouped_df[("train_overall_acc", k)]
	tmp_df[f"{k}_valid_overall_acc"] = grouped_df[("valid_overall_acc", k)]
	tmp_df[f"{k}_test_overall_acc"] = grouped_df[("test_overall_acc", k)]
	# tmp_df[f"{k}_train_matthews_corrcoef"] = grouped_df[("train_matthews_corrcoef", k)]
	tmp_df[f"{k}_valid_matthews_corrcoef"] = grouped_df[("valid_matthews_corrcoef", k)]
	tmp_df[f"{k}_test_matthews_corrcoef"] = grouped_df[("test_matthews_corrcoef", k)]
tmp_df["log_std_validation_loss"] = np.log10(tmp_df["std_validation_loss"])
tmp_df["1-std_validation_loss"] = 1-tmp_df["std_validation_loss"]
tmp_df["median_validation_training_loss_gap"] = tmp_df["median_validation_loss"] - tmp_df["median_training_loss"]
tmp_df["minimized_median_valid_matthews_corrcoef"] = 1-tmp_df["median_valid_matthews_corrcoef"] 
tmp_df["minimized_median_valid_overall_acc"] = 1-tmp_df["median_valid_overall_acc"] 
keys = [
	("num_neurons", -1),
	("mean_validation_loss", -1),
	# ("median_validation_loss", -1),
	("std_validation_loss", -1),
    # ("minimized_median_valid_matthews_corrcoef", -1),
    # ("minimized_median_valid_overall_acc", -1),
	# ("median_regularization_term", -1),
	# ("mean_valid_overall_acc", 1)
]
mcdm_keys = [k[0] for k in keys]
decision_matrix = tmp_df[mcdm_keys].to_numpy()
types = np.array([k[1] for k in keys])
pareto_efficient_mask = is_pareto_efficient(decision_matrix)

tmp_df = tmp_df.iloc[pareto_efficient_mask, :]
decision_matrix = tmp_df[mcdm_keys].to_numpy()
# print(decision_matrix[:10, :])

matrix = pymcdm.normalizations.normalize_matrix(decision_matrix, pymcdm.normalizations.minmax_normalization, types)
# pareto_decision_matrix[:, 1] = 1 - pareto_decision_matrix[:, 1]

# decision_matrix = decision_matrix[pareto_efficient_mask, :]
# tmp_df = tmp_df.iloc[pareto_efficient_mask, :]
# matrix = matrix[pareto_efficient_mask]
# matrix = pymcdm.normalizations.normalize_matrix(decision_matrix, pymcdm.normalizations.minmax_normalization, types)

# print(decision_matrix[:10, :])
weights=pymcdm.weights.equal_weights(matrix)
# weights=pymcdm.weights.entropy_weights(matrix)
weights=np.array([0.5, 0.4, 0.1])
# weights=np.array([1.0, 1.0, 1.0])
# weights=pymcdm.weights.entropy_weights(matrix)
# weights=pymcdm.weights.standard_deviation_weights(matrix)
# weights=pymcdm.weights.angle_weights(matrix, types)
# weights=pymcdm.weights.gini_weights(matrix, types)

mcdm_method = pymcdm.methods.ARAS()
mcdm_method = pymcdm.methods.COCOSO()
mcdm_method = pymcdm.methods.CODAS()
# mcdm_method = pymcdm.methods.COMET()
# mcdm_method = pymcdm.methods.MARCOS()
# mcdm_method = pymcdm.methods.EDAS() # bad
# mcdm_method = pymcdm.methods.MABAC() # bad
# mcdm_method = pymcdm.methods.OCRA() # failed
# mcdm_method = pymcdm.methods.MOORA() # failed
# mcdm_method = pymcdm.methods.COPRAS() # failed
mcdm_method = pymcdm.methods.COPRAS()
mcdm_method = pymcdm.methods.EDAS()
mcdm_method = pymcdm.methods.MABAC()
mcdm_method = pymcdm.methods.VIKOR()
mcdm_method = pymcdm.methods.MAIRCA()
mcdm_method = pymcdm.methods.TOPSIS(pymcdm.normalizations.vector_normalization)


# bounds =  np.vstack((
#     np.min(matrix, axis=0),
#     np.max(matrix, axis=0)
# )).T
# mcdm_method = pymcdm.methods.SPOTIS()
# tmp_df["rank"] = mcdm_method(matrix, weights, types, bounds=bounds)

# mcdm_method = pymcdm.methods.PROMETHEE_II('usual')

# cvalues =  np.vstack((
#     np.min(matrix, axis=0),
#     np.mean(matrix, axis=0),
#     np.max(matrix, axis=0)
# )).T
# mcdm_method = pymcdm.methods.COMET(cvalues=cvalues, rate_function=pymcdm.methods.COMET.topsis_rate_function(weights, types))

# mcdm_method = pymcdm.methods.TOPSIS()
# mcdm_method = pymcdm.methods.VIKOR()

# drop_idxs = np.where((matrix==0).sum(-1))[0]
# drop_idxs = (matrix==0).sum(-1).astype(np.bool)
# decision_matrix = decision_matrix[~drop_idxs, :]
# matrix = matrix[~drop_idxs, :]
# tmp_df = tmp_df.iloc[~drop_idxs, :]

# print(valid_idxs)
print((matrix==0).sum())

# mcdm_method = pymcdm.methods.MARCOS()
tmp_df["rank"] = mcdm_method(decision_matrix, weights, types)
tmp_df.head()


z = "mean_test_overall_acc"
z = "median_test_overall_acc"

y = "num_neurons"
# y = "median_valid_matthews_corrcoef"

# x = "median_regularization_term"
# x = "mean_regularization_term"
# x = "median_validation_loss"
# x = "std_validation_loss"
x = "rank"
# x = "median_valid_matthews_corrcoef"

color = "median_validation_training_loss_gap"
color = "median_validation_loss"
color = "mean_valid_overall_acc"
color = "median_valid_overall_acc"
# color="activation_name"
# color="median_validation_loss"
# color="is_pareto_efficient"
# color="rank"

# size = "1-std_validation_loss"
size="mean_valid_overall_acc"
size="num_neurons"


# tmp_df["is_pareto_efficient"] = is_pareto_efficient(decision_matrix)

# tmp_df["is_pareto_efficient"] = pareto_efficient_mask

# tmp_df=tmp_df[tmp_df["activation"] == "ReLU()"]
tmp_df.head()

# tmp_df.describe()

# ## KNEE ##
# from kneed import KneeLocator
# # tmp_df_pareto = tmp_df[tmp_df["is_pareto_efficient"]]
# tmp_df = tmp_df[tmp_df["activation"] == "Identity()"]
# k = KneeLocator(
# 	x=tmp_df["num_neurons"],
# 	# x=matrix[tmp_df["is_pareto_efficient"].to_numpy().astype(np.bool), 0],
# 	y=tmp_df["mean_validation_loss"],
# 	curve="convex",
# 	direction="decreasing",
# 	interp_method="polynomial",
# 	polynomial_degree=10,
# )
# import matplotlib.pyplot as plt
# # k.plot_knee_normalized()
# k.plot_knee()
# print(f"knee={k.knee} {k}")
# # # plt.show()

# ## KNEE ##
# # validation_df.describe()
# px.scatter(tmp_df, x="rank", y=z, color=color, size="num_neurons", height=500)
# px.scatter_3d(tmp_df[tmp_df["median_regularization_term"] <= tmp_df["median_regularization_term"].quantile(0.999)], x=x, y=y, z=z, color=color, size=size,height=800)
# px.scatter_3d(tmp_df[tmp_df['is_pareto_efficient']], x=x, y=y, z=z, color=color, size=size,height=800)
px.scatter_3d(tmp_df, x=x, y=y, z=z, color=color, size=size,height=800)
# px.scatter_3d(validation_df[validation_df["regularization_term"] <= validation_df["regularization_term"].quantile(0.999)], x="num_neurons", y="regularization_term", z="test_overall_acc", color="validation_loss", size="training_loss", height=800)
# px.line(tmp_df, x="num_neurons", y="median_validation_loss", log_y=True)
# sns.kdeplot(data=validation_df, x="regularization_term")
# tmp_df_pareto.sort_values(by=["rank"], ascending=False)

3


In [215]:
# px.scatter_3d(tmp_df, x="1-std_validation_loss", y="num_neurons", z="mean_test_overall_acc", color="mean_valid_overall_acc", size="mean_validation_loss",height=800)
matrix
# Faster than is_pareto_efficient_simple, but less readable.
def is_pareto_efficient(costs, return_mask = True):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :param return_mask: True to return a mask
    :return: An array of indices of pareto-efficient points.
        If return_mask is True, this will be an (n_points, ) boolean array
        Otherwise it will be a (n_efficient_points, ) integer array of indices.
    """
    is_efficient = np.arange(costs.shape[0])
    n_points = costs.shape[0]
    next_point_index = 0  # Next index in the is_efficient array to search for
    while next_point_index<len(costs):
        nondominated_point_mask = np.any(costs<costs[next_point_index], axis=1)
        nondominated_point_mask[next_point_index] = True
        is_efficient = is_efficient[nondominated_point_mask]  # Remove dominated points
        costs = costs[nondominated_point_mask]
        next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
    if return_mask:
        is_efficient_mask = np.zeros(n_points, dtype = bool)
        is_efficient_mask[is_efficient] = True
        return is_efficient_mask
    else:
        return is_efficient

pareto_idxs = is_pareto_efficient(matrix)
matrix[pareto_idxs]


array([[0.66  , 0.5932, 0.0845, 0.8542],
       [0.65  , 0.5829, 0.1159, 0.898 ],
       [0.64  , 0.6678, 0.1337, 0.8776],
       [0.62  , 0.627 , 0.0989, 0.8907],
       [0.53  , 0.4737, 0.1935, 0.9592],
       [0.33  , 0.5662, 0.361 , 0.9767],
       [0.99  , 0.2272, 0.    , 0.7099],
       [0.75  , 0.1221, 0.4008, 0.9111],
       [0.72  , 0.1717, 0.4074, 0.9052],
       [0.65  , 0.    , 0.3178, 0.9213],
       [0.63  , 0.0296, 0.3172, 0.9417],
       [0.59  , 0.1855, 0.1815, 0.8746],
       [0.57  , 0.09  , 0.2458, 0.9402],
       [0.55  , 0.5742, 0.1328, 0.9402],
       [0.54  , 0.4789, 0.1393, 0.9329],
       [0.53  , 0.5491, 0.1487, 0.8367],
       [0.51  , 0.3393, 0.2379, 0.9417],
       [0.48  , 0.4887, 0.1633, 0.9227],
       [0.46  , 0.4311, 0.2161, 0.8878],
       [0.42  , 0.2378, 0.2611, 0.9067],
       [0.39  , 0.54  , 0.2869, 0.9461],
       [0.38  , 0.5559, 0.3191, 0.9548],
       [0.35  , 0.4223, 0.3339, 0.9708],
       [0.34  , 0.4723, 0.32  , 0.9665],
       [0.32  , 